# PyGrid Client Association Requests API

### Import Libs

In [1]:
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient

import syft as sy
import torch as th

sy.VERBOSE = False

### Connect performing authentication process on both nodes

In [2]:
domain_node = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"admin@email.com", "password":"pwd123"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
)

network_node = connect(
    url="http://localhost:5001", # Network Address
    credentials={"email":"admin@email.com", "password":"pwd123"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
)

### Create an association request on Node1 to node2

In [ ]:
domain_node.association_requests.create(
    name="OpenMined Network Association Request",
    address="http://localhost:5001",
    sender_address="http://localhost:5000")

### Check association requests on Node 1

In [ ]:
domain_node.association_requests.all(pandas=True)

### Check association requests on Node 2

In [ ]:
network_node.association_requests.all(pandas=True)

### Accept/Deny Association Request on Node 2

In [ ]:
network_node.association_requests[1].accept()

In [ ]:
domain_node.association_requests.all(pandas=True)

In [ ]:
network_node.association_requests.all(pandas=True)

### Check if association request status were updated on both nodes

In [ ]:
assert domain_node.association_requests[1].accepted == True
assert domain_node.association_requests[1].pending == False
assert network_node.association_requests[1].accepted == True
assert network_node.association_requests[1].pending == False

### Broadcast Search

Use this association to perform reach the domain by performing broadcast queries through the network

In [3]:
import torch as th

x = th.tensor([1,2,3,4,5,6,7,8,9])
x_s = x.send(domain_node, pointable=True, tags=['#diabetes', "x"])

In [5]:
network_node.search(query=["#diabetes"],pandas=True)

,match-nodes
0,http://localhost:5000
